이번에는 'Visdom'에 대해서 정리해보겠습니다.

정보는
* https://github.com/facebookresearch/visdom
* https://github.com/GunhoChoi/PyTorch-FastCampus/tree/master/02_Regression%26NN

참고하시면 좋을 것 같습니다.

Linux에서 Visdom을 설치하는 방법은 다음과 같습니다.
- pip install visdom

Visdom을 실행하는 방법은 다음과 같습니다.
- python -m visdom.server

# Visdom으로 할 수 있는 것들

### Basics

Visdom offers the following basic visualization funcions:
- vis.image : image
- vis.images : list of images
- vis.text : arbitrary HTML
- vis.audio : audio
- vis.video : videos
- vis.svg : SVG object
- vis.matplot : matplotlib plot
- vis.save : serialize state server-side

### Plotting

We have wrapped several common plot types to make creating basic visualizations easily. These visualizations are powered by Plotly. The following API is currently supported:
- vis.scatter : 2D or 3D scatter plots
- vis.line : line plots
- vis.updateTrace : update existing line / scatter plots
- vis.stem : stem plots
- vis.heatmap : heatmap plots
- vis.bar : bar graphs
- vis.histogram : histograms
- vis.boxplot : boxplots
- vis.surf : surface plots
- vis.contour : contour plots
- vis.quiver : quiver plots
- vis.mesh : mesh plots

이번 자료에서는 **Basics와 Plotting에서 vis.line** 만 정리를 할 것입니다.

먼저 기본 import부터 작성하겠습니다.

In [1]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable 
from torch.utils.data import Dataset, DataLoader 

import numpy as np 
import visdom # visdom을 사용하려면 다음과 같이 import를 하면 됩니다.

import torchvision 
import torchvision.transforms as transforms 
import torchvision.datasets as datasets

# import matplotlib 
# import matplotlib.pyplot as plt 

그리고 visdom을 사용하기 편하게 변수로 설정하겠습니다.

In [2]:
vis = visdom.Visdom()

***

# Basics

### Text

In [3]:
textwindow = vis.text('Hello Pytorch')

띄어둔 Local host에 가시면 visdom을 통해 출력이 된 것을 보실 수 있습니다.

### Image & Images

In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.MNIST(root='./mnist_data',
                                                            train=True,
                                                            download=True,
                                                            transform=transform)
 
testset = torchvision.datasets.MNIST(root='./mnist_data',
                                                            train=False,
                                                            download=True,
                                                            transform=transform)

trainloader = DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=8, shuffle=False, num_workers=2)

visdom으로 Image를 확인해보겠습니다.

In [5]:
for i, data in enumerate(trainloader):
    img, label = data
    vis.image(img[0]) # Image 하나만 보는 방법
    vis.images(img) # Image들을 다 보는 방법
    break

In [6]:
for i, data in enumerate(testloader):
    img, label = data
    vis.image(img[0])
    vis.images(img)
    break

***

# Line plot & Line plot Update in Plotting

### Line plot을 어떻게 그릴까요?

Y값만 넣어서 그래프를 그려보겠습니다. 그래프를 보면 Y값만 넣었기 때문에 **X값은 [0,1]사이** 로 정의가 됩니다.

In [7]:
plt = vis.line(Y=torch.randn(5))

이번에는 Y값과 X값을 같이 넣어서 그래프를 그려보겠습니다. X를 [0,1,2,3,4]로 넣었기 때문에 적절한 값이 그려지는 것을 보실 수 있습니다.

In [8]:
plot = vis.line(Y=torch.randn(5), X=np.array([0,1,2,3,4]))

### Line plot은 어떻게 업데이트를 할까요?

In [9]:
vis.line(Y=torch.randn(1), X=np.array([5]), win=plot, update='append')

'window_362436b4c36324'

In [8]에서 그렸던 plot을 업데이트할 변수명 "win=plot"으로 넣어주시고, "update='append'"으로 적어주시면

X축 5번에 새로운 그래프가 그려지는 것을 보실 수 있습니다.

### 계속 업데이트는 어떻게 할까요?

In [10]:
for i in range(5000):
    vis.line(Y=torch.randn(1), X=np.array([i+5]), win=plot, update='append')

아까 4번까지는 그렸기 때문에 'i+5'로 하여 range를 5000까지 나타내보겠습니다. visdom으로 가시면 5000까지 그래프가 그려지는 것을 보실 수 있습니다.

### 하나의 Line plot에 두 개 이상의 그래프 그리기

두 개의 그래프를 그리고 싶다면 'Y=torch.randn(10, **2** )'를 해주시고, column_stack이라는 것을 이용하여 그릴 것을 두 가지를 넣어주시면 됩니다.

In [11]:
vis.line(Y=torch.randn(10,2), X=np.column_stack((np.arange(0, 10), np.arange(0, 10))))

'window_3624374bce72f2'

세 개의 그래프는 아래와 같이 적으면 되겠죠?

In [12]:
vis.line(Y=torch.randn(10,3), X=np.column_stack((np.arange(0, 10), np.arange(0, 10), np.arange(0, 10))))

'window_3624374dedd65c'

### 그래프 형태 조작 및 정보 추가

'opts=dict()'에서 괄호안에 그래프의 정보를 추가할 수 있습니다. 또한 dict안에 'showlegend'를 적으면 표의 범주(범례)를 보일 것인지 말 것인지를 정할 수 있습니다.

In [13]:
vis.line(Y=torch.randn(10,2), X=np.column_stack((np.arange(0, 10), np.arange(0, 10))), opts=dict(title='hello', showlegend=True))

'window_3624378af8dee8'

'legend'에서 이름까지 보이게 하고 싶으면 다음과 같이 입력하면 됩니다.

In [14]:
vis.line(Y=torch.randn(10,2), X=np.column_stack((np.arange(0, 10), np.arange(0, 10))), 
         opts=dict(title='hello',
                   legend=['1번', '2번'],
                   showlegend=True))

'window_36243791f64b3c'